In [ ]:
import requests
import json

class PaystackPaymentProcessor:
    def __init__(self, secret_key):
        """
        Initialize the PayStack payment processor
        
        Args:
            secret_key (str): Your PayStack secret key
        """
        self.base_url = "https://api.paystack.co"
        self.secret_key = secret_key
        self.headers = {
            "Authorization": f"Bearer {self.secret_key}",
            "Content-Type": "application/json"
        }

    def initialize_transaction(self, customer_details):
        """
        Initialize a new transaction with PayStack
        
        Args:
            customer_details (dict): Dictionary containing transaction details
            
        Example customer_details:
        {
            "email": "customer@example.com",
            "amount": 50000,  # Amount in kobo (500 NGN)
            "callback_url": "https://yourwebsite.com/payment/callback",
            "order_id": "ORD-2024-001"
        }
        
        Returns:
            dict: Transaction initialization response
        """
        # Prepare the transaction payload
        payload = {
            "email": customer_details["email"],
            "amount": str(customer_details["amount"]),  # Must be a string
            "callback_url": customer_details.get("callback_url", ""),
            "metadata": {
                "custom_fields": [
                    {
                        "display_name": "Order ID",
                        "variable_name": "order_id",
                        "value": customer_details.get("order_id", "")
                    }
                ]
            }
        }

        try:
            # Send POST request to PayStack API
            response = requests.post(
                f"{self.base_url}/transaction/initialize", 
                headers=self.headers, 
                data=json.dumps(payload)
            )
            
            # Parse the response
            response_data = response.json()
            
            # Check if transaction initialization was successful
            if response_data.get("status") == True:
                return {
                    "success": True,
                    "authorization_url": response_data["data"]["authorization_url"],
                    "reference": response_data["data"]["reference"]
                }
            else:
                return {
                    "success": False,
                    "error": response_data.get("message", "Unknown error")
                }
        
        except requests.exceptions.RequestException as e:
            # Handle network-related errors
            return {
                "success": False,
                "error": f"Network error: {str(e)}"
            }

    def verify_transaction(self, reference):
        """
        Verify a completed transaction
        
        Args:
            reference (str): Transaction reference code
        
        Returns:
            dict: Transaction verification details
        """
        try:
            response = requests.get(
                f"{self.base_url}/transaction/verify/{reference}",
                headers=self.headers
            )
            
            response_data = response.json()
            
            if response_data.get("status") == True:
                transaction_data = response_data["data"]
                return {
                    "success": True,
                    "amount": transaction_data["amount"],
                    "currency": transaction_data["currency"],
                    "status": transaction_data["status"],
                    "reference": transaction_data["reference"]
                }
            else:
                return {
                    "success": False,
                    "error": response_data.get("message", "Verification failed")
                }
        
        except requests.exceptions.RequestException as e:
            return {
                "success": False,
                "error": f"Network error: {str(e)}"
            }

# Example Usage
def main():
    # Replace with your actual PayStack secret key
    SECRET_KEY = "sk_test_your_secret_key_here"
    
    # Create PayStack processor instance
    paystack = PaystackPaymentProcessor(SECRET_KEY)
    
    # Example customer transaction
    customer_transaction = {
        "email": "customer@example.com",
        "amount": 50000,  # 500 NGN (amount in kobo)
        "callback_url": "https://yourwebsite.com/payment/callback",
        "order_id": "ORD-2024-001"
    }
    
    # Initialize transaction
    init_result = paystack.initialize_transaction(customer_transaction)
    
    if init_result["success"]:
        print("Transaction Initialized:")
        print(f"Authorization URL: {init_result['authorization_url']}")
        print(f"Reference: {init_result['reference']}")
        
        # Simulate transaction verification (would typically happen after customer pays)
        verify_result = paystack.verify_transaction(init_result['reference'])
        
        if verify_result["success"]:
            print("\nTransaction Verified:")
            print(f"Amount: {verify_result['amount']}")
            print(f"Status: {verify_result['status']}")
        else:
            print("\nVerification Failed:")
            print(f"Error: {verify_result['error']}")
    else:
        print("Transaction Initialization Failed:")
        print(f"Error: {init_result['error']}")

# Protect the script from auto-execution when imported
if __name__ == "__main__":
    main()

In [ ]:
import pandas as pd
import numpy as np

# Sample transaction data
transactions = pd.DataFrame({
    'amount': [100, 5000, 250, 10000, 750],
    'customer_history': [50, 3, 27, 1, 12],
    'time_of_day': [14, 3, 16, 2, 13],
    'unusual_location': [0, 1, 0, 1, 0]
})

# Risk weights
weights = {
    'amount': 0.3,
    'customer_history': 0.3,
    'time_of_day': 0.2,
    'unusual_location': 0.2
}

# Normalize features
def normalize(series):
    return (series - series.min()) / (series.max() - series.min())

# Calculate risk score (higher = riskier)
transactions['amount_risk'] = normalize(transactions['amount']) * weights['amount']
transactions['history_risk'] = (1 - normalize(transactions['customer_history'])) * weights['customer_history']
transactions['time_risk'] = (abs(transactions['time_of_day'] - 12) / 12) * weights['time_of_day']
transactions['location_risk'] = transactions['unusual_location'] * weights['unusual_location']

transactions['total_risk'] = (
    transactions['amount_risk'] + 
    transactions['history_risk'] + 
    transactions['time_risk'] + 
    transactions['location_risk']
)

print(transactions[['amount', 'total_risk']])